# Recommender System for Video Games using IGDB + PyTorch

This notebook demonstrates a simple pipeline to:
- Fetch game metadata from IGDB API
- Build a toy interaction dataset
- Train a basic Two-Tower recommendation model using PyTorch
- Generate recommendations

⚠️ You must replace CLIENT_ID and ACCESS_TOKEN with your own IGDB credentials.

In [ ]:

# Install required libraries (if needed)
# !pip install requests torch pandas scikit-learn


## 1. IGDB API Connection

In [ ]:

import requests
import pandas as pd

CLIENT_ID = "dth2x0rwoi4vy8sf33vwg7fb4lc1g5"
ACCESS_TOKEN = "55cncv2eib6ggcwrr0652i1qhtl7dy"

headers = {
    "Client-ID": CLIENT_ID,
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

def fetch_games():
    url = "https://api.igdb.com/v4/games"
    query = "fields id,name,genres.name,platforms.name; limit 50;"
    response = requests.post(url, headers=headers, data=query)
    return response.json()

games = fetch_games()
print("Fetched games:", len(games))


Fetched games: 50


[{'id': 350392,
  'genres': [{'id': 5, 'name': 'Shooter'}],
  'name': 'Rival Species',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}]},
 {'id': 30492,
  'genres': [{'id': 12, 'name': 'Role-playing (RPG)'},
   {'id': 15, 'name': 'Strategy'},
   {'id': 31, 'name': 'Adventure'},
   {'id': 32, 'name': 'Indie'}],
  'name': 'Hearts of Chaos',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}]},
 {'id': 339266,
  'name': 'Power Guy World',
  'platforms': [{'id': 19, 'name': 'Super Nintendo Entertainment System'}]},
 {'id': 371776,
  'genres': [{'id': 31, 'name': 'Adventure'},
   {'id': 32, 'name': 'Indie'},
   {'id': 34, 'name': 'Visual Novel'}],
  'name': 'Born for the Light: Wavelength Radiant Collapse',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}]},
 {'id': 376989,
  'genres': [{'id': 10, 'name': 'Racing'}],
  'name': 'Dune Buggy',
  'platforms': [{'id': 82, 'name': 'Web browser'}]},
 {'id': 85727,
  'genres': [{'id': 12, 'name': 'Role-playing (RPG)'

## 2. Build Game Feature Table

In [5]:

records = []
for g in games:
    genres = [x['name'] for x in g.get('genres', [])]
    platforms = [x['name'] for x in g.get('platforms', [])]
    records.append({
        "game_id": g["id"],
        "name": g["name"],
        "genres": genres,
        "platforms": platforms
    })

df_games = pd.DataFrame(records)
df_games.head()


,game_id,name,genres,platforms
0,350392,Rival Species,[Shooter],[PC (Microsoft Windows)]
1,30492,Hearts of Chaos,"[Role-playing (RPG), Strategy, Adventure, Indie]",[PC (Microsoft Windows)]
2,339266,Power Guy World,[],[Super Nintendo Entertainment System]
3,371776,Born for the Light: Wavelength Radiant Collapse,"[Adventure, Indie, Visual Novel]",[PC (Microsoft Windows)]
4,376989,Dune Buggy,[Racing],[Web browser]


## 3. Create Synthetic Interaction Dataset

In [8]:

import random

users = list(range(1, 21))
interactions = []

for u in users:
    liked = random.sample(df_games["game_id"].tolist(), 5)
    for g in liked:
        interactions.append([u, g, 1])

df_inter = pd.DataFrame(interactions, columns=["user_id", "game_id", "label"])
df_inter.head()


,user_id,game_id,label
0,1,41420,1
1,1,262483,1
2,1,359862,1
3,1,371776,1
4,1,325939,1


## 4. Encode Users and Games

In [9]:

import torch
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
game_enc = LabelEncoder()

df_inter["user"] = user_enc.fit_transform(df_inter["user_id"])
df_inter["game"] = game_enc.fit_transform(df_inter["game_id"])

num_users = df_inter["user"].nunique()
num_games = df_inter["game"].nunique()

num_users, num_games


(20, 43)

## 5. Define Two-Tower Model in PyTorch

In [10]:

import torch.nn as nn
import torch.nn.functional as F

class TwoTower(nn.Module):
    def __init__(self, n_users, n_games, emb_dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.game_emb = nn.Embedding(n_games, emb_dim)

    def forward(self, user, game):
        u = self.user_emb(user)
        g = self.game_emb(game)
        score = (u * g).sum(dim=1)
        return torch.sigmoid(score)

model = TwoTower(num_users, num_games)


## 6. Train the Model

In [11]:

from torch.utils.data import DataLoader, TensorDataset

X_users = torch.tensor(df_inter["user"].values)
X_games = torch.tensor(df_inter["game"].values)
y = torch.tensor(df_inter["label"].values, dtype=torch.float32)

dataset = TensorDataset(X_users, X_games, y)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCELoss()

for epoch in range(10):
    total = 0
    for u, g, l in loader:
        optimizer.zero_grad()
        preds = model(u, g)
        loss = loss_fn(preds, l)
        loss.backward()
        optimizer.step()
        total += loss.item()
    print(f"Epoch {epoch}: loss={total:.4f}")


Epoch 0: loss=28.0349
Epoch 1: loss=20.4295
Epoch 2: loss=15.5549
Epoch 3: loss=11.0583
Epoch 4: loss=7.4831
Epoch 5: loss=5.4188
Epoch 6: loss=2.8359
Epoch 7: loss=1.7021
Epoch 8: loss=1.1482
Epoch 9: loss=0.6461


## 7. Generate Recommendations

In [12]:

def recommend_for_user(user_id, top_k=5):
    u = torch.tensor([user_enc.transform([user_id])[0]])
    scores = []
    for g in range(num_games):
        g_t = torch.tensor([g])
        s = model(u, g_t).item()
        scores.append((g, s))

    scores.sort(key=lambda x: x[1], reverse=True)
    top = scores[:top_k]

    recs = []
    for game_idx, sc in top:
        game_real = game_enc.inverse_transform([game_idx])[0]
        name = df_games[df_games.game_id == game_real].name.values[0]
        recs.append((name, sc))
    return recs

print(recommend_for_user(1))


[('Caves of Qud', 0.9999994039535522), ('Street Vendor Simulator', 0.9999977350234985), ('Death of the Reprobate', 0.9994179010391235), ('Born for the Light: Wavelength Radiant Collapse', 0.9994157552719116), ('Sonic Uprising', 0.9993174076080322)]


## Done!
This notebook shows a minimal working example. In a real project you would:
- Use real interaction data
- Add negative sampling
- Add game content features
- Train on larger datasets